In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

In [15]:
# Configurer les options pour le mode headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Activer le mode headless
chrome_options.add_argument("--disable-gpu")  # Nécessaire pour certains environnements
chrome_options.add_argument("--window-size=1920x1080")  # Définir une taille de fenêtre

# Initialiser le driver avec les options
driver = webdriver.Chrome(options=chrome_options)

# Charger l'URL
driver.get("https://www.oddsportal.com/value-bets/#1/0/overall")

# Récupérer le contenu HTML après exécution de JavaScript
html = driver.page_source

# Analyser avec BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Sauvegarder le contenu HTML dans un fichier local
with open("page_content.html", "w", encoding="utf-8") as file:
    file.write(html)

print("Le contenu HTML a été sauvegardé dans 'page_content.html'.")

Le contenu HTML a été sauvegardé dans 'page_content.html'.


In [16]:
# Trouver la balise <title>
title = soup.find('title')
print(title.text)

# Fermer le driver
driver.quit()

Value Odds - Value Bets | Odds Portal
